In [2]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.deprecated.doc2vec import LabeledSentence
from gensim.models.word2vec import Word2Vec
from gensim.models.phrases import Phraser, Phrases
from gensim.parsing.porter import PorterStemmer
from gensim.parsing.preprocessing import STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords
from string import digits

import pandas as pd
import numpy as np
import string
import re
import random 
import os
import csv
import pickle

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
nltk.download('stopwords')

from sklearn import metrics
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier 
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix, precision_recall_curve, plot_precision_recall_curve, auc, average_precision_score,classification_report, confusion_matrix, accuracy_score, average_precision_score, precision_score, f1_score, recall_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.utils.multiclass import unique_labels
from sklearn.model_selection import cross_val_score, cross_validate, RepeatedStratifiedKFold, train_test_split,KFold, cross_val_score, GridSearchCV
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier

from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from modAL.uncertainty import entropy_sampling
from modAL.density import information_density

from scipy.stats import entropy
from matplotlib import pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yuchenluo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# PREPARE DATA

In [8]:
# ## load labelled data
# clas_dat1 = pd.read_csv("/Users/carlyknight/Dropbox/PROJECTS/Forecasting Downturns/data/random_sample_CRK_YL-6-7-21_v1.csv")

# # keep relavant rows and columns
# clas_dat1 = clas_dat1[['id', 'text', 'final_code']]
# clas_dat2 = pd.read_csv("/Users/carlyknight/Dropbox/PROJECTS/Forecasting Downturns/data/random_sample3_CRK_v1.csv")
# clas_dat2 = clas_dat2[['id', 'text',"include"]]
# clas_dat2.columns = clas_dat1.columns
# clas_dat3 = pd.read_csv("/Users/carlyknight/Dropbox/PROJECTS/Forecasting Downturns/data/random_sample2_YL_v1.csv")
# clas_dat3 = clas_dat3[['id', 'text',"match_yl"]]
# clas_dat3.columns = clas_dat1.columns
# clas_dat = clas_dat1.append([clas_dat2, clas_dat3])

clas_dat = pd.read_csv("coded_sample.csv")

In [3]:
#clas_dat1.shape(300)
#clas_dat2.shape (293)
#clas_dat3.shape (299)

In [4]:
#check for duplicates: none
clas_dat = clas_dat.drop_duplicates()
clas_dat.shape

(897, 3)

In [11]:
#### preprocessing -------------------------------
punctuation_dictionary = {s:None for s in list(string.punctuation)}

punctuation_translator = str.maketrans(punctuation_dictionary)

stop_words = set(stopwords.words('english'))

# (remove punctuation, numbers, lowercase, stop words)
def text_cleaner_all(text, punctuation_translator):
    text = text.replace('c("', '')
    text = str(text).translate(punctuation_translator)
    text = text.lower()
    remove_digits = str.maketrans('', '', digits)
    text = text.translate(remove_digits)
    word_tokens = word_tokenize(text)
    filtered_text = [w for w in word_tokens if not w.lower() in stop_words]
    text = ' '.join(filtered_text)
    return(text)

# (remove punctuation, lowercase, stop words)
def text_cleaner_mod(text, punctuation_translator):
    text = text.replace('c("', '')
    text = str(text).translate(punctuation_translator)
    text = text.lower()
    word_tokens = word_tokenize(text)
    filtered_text = [w for w in word_tokens if not w.lower() in stop_words]
    text = ' '.join(filtered_text)
    return(text)

# (remove punctuation, lowercase)
def text_cleaner_min(text, punctuation_translator):
    text = text.replace('c("', '')
    text = str(text).translate(punctuation_translator)
    text = text.lower()
    return(text)


### Pick a level of preprocessing

In [13]:
clas_dat["clean_text"] = clas_dat["text"].apply(lambda x: text_cleaner_all(x, punctuation_translator))

In [7]:
# find phrases
phrases1 = Phrases(map(lambda x: x.split(), clas_dat["clean_text"].tolist())) #bigram
phrases2 = Phrases(phrases1[map(lambda x: x.split(), clas_dat["clean_text"].tolist())]) #trigram
clas_dat["phrased_text"] = clas_dat["clean_text"].apply(lambda x: " ".join(phrases2[phrases1[x.split()]]))

In [14]:
clas_dat.yl_code.value_counts()

0    614
1    586
Name: yl_code, dtype: int64

# Logistic Regression Classifier (BOW+ Tf-Idf)


In [22]:
### Moderate text cleaning (remove punctuation, lowercase, stop words) --------------------------
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( max_features = 5000, min_df=5)
X = vectorizer.fit_transform(clas_dat["clean_text"]).toarray()

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
tfidfconverter = TfidfTransformer()
X_tf = tfidfconverter.fit_transform(X).toarray()

y = np.array(clas_dat['yl_code'])


In [10]:
accuracy_scores= []
precision_scores = []
recall_scores = []

for i in range(0,20):
    ## load the test set 
    X_train, X_test, y_train, y_test = train_test_split(X_tf, y, test_size=0.2)
    classifier = LogisticRegression(C= 0.1, penalty= 'l2', solver= 'liblinear').fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
#     print(confusion_matrix(y_test,y_pred))
    ## Predict out-of-sample on the test set and compute accuracy
    accuracy_scores= accuracy_scores+ [accuracy_score(y_test, y_pred)]
    # precision tp / (tp + fp)
    precision = precision_score(y_test, y_pred)
    precision_scores= precision_scores+ [precision]
#     print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(y_test, y_pred)
    recall_scores= recall_scores+ [recall]
#     print('Recall: %f' % recall)

print("Mean Accuracy: " + str(np.mean(accuracy_scores)))
print("Average precision: " + str(np.mean(precision_scores)))
print("Average recall: " + str(np.mean(recall_scores)))

Mean Accuracy: 0.6852777777777778
Average precision: 0.7103249117744753
Average recall: 0.7021262061464687


In [3]:
all_text = pd.read_csv("/Users/yuchenluo/Dropbox/Forecasting Downturns/data/all_text.csv")

In [4]:
all_text.head()

,id,text
0,7VS1-X791-2R2Y-70SK-00000-00.txt,"c(""May 23--Record demand for area food pantry ..."
1,4VGG-NH20-TX12-N12V-00000-00.txt,"c(""Jan. 28--STOCKTON -- All but six of the 189..."
2,7VTR-NPY0-Y9J0-Y26V-00000-00.txt,"c(""May 31--It's a simple question: What are yo..."
3,4VG1-37N0-TX12-N0MF-00000-00.txt,"c(""Jan. 25--This is the best time for buying p..."
4,7WDR-PFY0-Y9J0-Y03G-00000-00.txt,"c(""Aug. 17--GLENS FALLS -- The Glens Falls Far..."


### Cross-validated scores & selection based on accuracy + precision

gets highest precision but low recall

In [25]:
# focus on accuracy and precision 
#https://medium.com/@venkatasujit272/overview-of-cross-validation-3785d5414ece
#https://datascience.stackexchange.com/questions/21877/how-to-use-the-output-of-gridsearch

#training set
X_train, X_test, y_train, y_test = train_test_split(X_tf, y, test_size=0.2)

model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
scoring = ['accuracy', 'precision']

grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats = 5, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring= ['accuracy', 'precision'],refit = "accuracy")
grid_result = grid_search.fit(X_train, y_train)

# summarize results
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
precisions = grid_result.cv_results_['mean_test_precision']
accuracys =  grid_result.cv_results_['mean_test_accuracy']
std_prec = grid_result.cv_results_['std_test_precision']
std_acc = grid_result.cv_results_['std_test_accuracy']
params = grid_result.cv_results_['params']

for prec, acc, param in zip(precisions, accuracys, params):
    print("Precision: %f (Accuracy: %f) with: %r" % (prec, acc, param)) 

Best Accuracy: 0.743125 using {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
Precision: 0.737326 (Accuracy: 0.729792) with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
Precision: 0.737326 (Accuracy: 0.729792) with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
Precision: 0.736974 (Accuracy: 0.729583) with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Precision: 0.751062 (Accuracy: 0.738125) with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
Precision: 0.751062 (Accuracy: 0.738125) with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
Precision: 0.751284 (Accuracy: 0.738750) with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
Precision: 0.770129 (Accuracy: 0.743125) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
Precision: 0.770129 (Accuracy: 0.743125) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
Precision: 0.768164 (Accuracy: 0.742500) with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
Precision: 0.793391 (Accuracy: 0.723750) with: {'C':

In [ ]:
#https://stackoverflow.com/questions/42362027/model-help-using-scikit-learn-when-using-gridsearch/42364900#42364900

In [17]:
y_pred = grid_search.best_estimator_.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))

Accuracy:  0.7166666666666667
Precision:  0.7333333333333333
Recall:  0.6581196581196581
F1:  0.6936936936936937


In [27]:

import joblib
joblib.dump(grid_search.best_estimator_, 'best_estimator0816.pkl')

['best_estimator0816.pkl']

### Cross-validated scores & selection based on f1 score

Gets highest overall model

In [28]:
# focus on accuracy and precision 
#https://medium.com/@venkatasujit272/overview-of-cross-validation-3785d5414ece
#https://datascience.stackexchange.com/questions/21877/how-to-use-the-output-of-gridsearch

#training set
X_train, X_test, y_train, y_test = train_test_split(X_tf, y, test_size=0.2)

model = LogisticRegression()
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

# define grid search
grid = dict(solver=solvers,penalty=penalty,C=c_values)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats = 5, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=grid, n_jobs=-1, cv=cv, scoring= 'f1_micro')
grid_result = grid_search.fit(X_train, y_train)


# summarize results
print("Best F1: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
f1s = grid_result.cv_results_['mean_test_score']
params = grid_result.cv_results_['params']

for f1, param in zip(f1s, params):
    print("F1: %f with: %r" % (f1, param)) 


Best F1: 0.715501 using {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
F1: 0.696530 with: {'C': 100, 'penalty': 'l2', 'solver': 'newton-cg'}
F1: 0.696530 with: {'C': 100, 'penalty': 'l2', 'solver': 'lbfgs'}
F1: 0.696256 with: {'C': 100, 'penalty': 'l2', 'solver': 'liblinear'}
F1: 0.705196 with: {'C': 10, 'penalty': 'l2', 'solver': 'newton-cg'}
F1: 0.705196 with: {'C': 10, 'penalty': 'l2', 'solver': 'lbfgs'}
F1: 0.704632 with: {'C': 10, 'penalty': 'l2', 'solver': 'liblinear'}
F1: 0.714937 with: {'C': 1.0, 'penalty': 'l2', 'solver': 'newton-cg'}
F1: 0.714937 with: {'C': 1.0, 'penalty': 'l2', 'solver': 'lbfgs'}
F1: 0.715501 with: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}
F1: 0.707938 with: {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
F1: 0.707938 with: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
F1: 0.708517 with: {'C': 0.1, 'penalty': 'l2', 'solver': 'liblinear'}
F1: 0.586553 with: {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
F1: 0.586553 with: {'C': 0.01, 'p

In [29]:
y_pred = grid_search.best_estimator_.predict(X_test)

print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))

Accuracy:  0.75
Precision:  0.7472527472527473
Recall:  0.7555555555555555
F1:  0.7513812154696132


In [30]:
#save
import joblib
joblib.dump(grid_search.best_estimator_, '/Users/carlyknight/Dropbox/PROJECTS/Forecasting Downturns/data/best_estimator_f1score.pkl')

['/Users/carlyknight/Dropbox/PROJECTS/Forecasting Downturns/data/best_estimator_f1score.pkl']

### SGD



In [33]:
#training set
X_train, X_test, y_train, y_test = train_test_split(X_tf, y, test_size=0.2)

model = SGDClassifier(max_iter= 1000)
params = {
    "loss" : ["hinge", "log", "squared_hinge", "modified_huber"],
    "alpha" : [0.0001, 0.001, 0.01, 0.1],
    "penalty" : ["l2", "l1", "none"],
}

# define grid search
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats = 5, random_state=1)
grid_search = GridSearchCV(estimator=model, param_grid=params, cv=cv, scoring= "f1_micro")
grid_result = grid_search.fit(X_train, y_train)

# summarize results
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
precisions = grid_result.cv_results_['mean_test_precision']
accuracys =  grid_result.cv_results_['mean_test_accuracy']
std_prec = grid_result.cv_results_['std_test_precision']
std_acc = grid_result.cv_results_['std_test_accuracy']
params = grid_result.cv_results_['params']

for prec, acc, param in zip(precisions, accuracys, params):
    print("Precision: %f (Accuracy: %f) with: %r" % (prec, acc, param)) 

/Users/carlyknight/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/carlyknight/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/carlyknight/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/carlyknight/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  Conv

/Users/carlyknight/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/carlyknight/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/carlyknight/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/carlyknight/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  Conv

/Users/carlyknight/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/carlyknight/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/carlyknight/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/Users/carlyknight/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  Conv

/Users/carlyknight/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_stochastic_gradient.py:577: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


KeyboardInterrupt: 

In [ ]:
# know the data
# descriptives
# over time, party, how much we lost, word count
# when the keywords appear 
# one-pager on classifier performance 
# sentiment analysis (dictionary)
# POS (who animnates the recession)
# three articles on POS 